In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.pipeline import FeatureUnion
from nltk.corpus import stopwords 
import os
import string

import heapq 

In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def removeStopWords(wordList):
    return [word for word in wordList if word not in stopwords.words('english')]

# DO NOT USE
def calcCosSimOld(aMat,bVec):
    # Calculate cosine simularity
    a = np.sum(aMat,axis=0)/aMat.shape[0]
    b = bVec.copy()
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def calcCosSim(a,b):
    # Calculate cosine simularity
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def removeStem(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    tmpStr = ''
    for w in words:
        tmpStr += ps.stem(w) + ' '
    return tmpStr

In [13]:
rootPath = './hotel/'
complaintSubPath = 'hotelMain/'

contractPath = './hotel_policy.txt.'
#contractPath = rootPath + '/termService/' + 'related.txt'
#contractPath = rootPath + '/termService/' + 'unrelated.txt'


#contractTxt = open(contractPath, encoding="utf8", errors='ignore').read()
contractTxt = 'I am suffering from bleeding gums covered with United Healthcare'


contractVec = removeStopWords([removeStem(contractTxt)])

vec = CountVectorizer()
contractFreq = vec.fit_transform(contractVec)


compList = []

# Used for removing stem from words

'''
for cFilePath in os.listdir(rootPath + complaintSubPath):
    cFilePath = rootPath + complaintSubPath + cFilePath
    cFileTxt = open(cFilePath,'r').read()

    cFileTxt = removeStem(cFileTxt)
    
    compList.append(cFileTxt + ' ')


compList = [''.join(str(v) for v in compList)]  # We are combining all complaints into one
complantCnt = CountVectorizer()
a = complantCnt.fit_transform(compList)
'''
#print(contractVec)

"\nfor cFilePath in os.listdir(rootPath + complaintSubPath):\n    cFilePath = rootPath + complaintSubPath + cFilePath\n    cFileTxt = open(cFilePath,'r').read()\n\n    cFileTxt = removeStem(cFileTxt)\n    \n    compList.append(cFileTxt + ' ')\n\n\ncompList = [''.join(str(v) for v in compList)]  # We are combining all complaints into one\ncomplantCnt = CountVectorizer()\na = complantCnt.fit_transform(compList)\n"

In [54]:
customerTxt = 'I am suffering from bleeding gums covered with United Healthcare'
customerVec = removeStopWords([removeStem(customerTxt)])


#doctorTxt = 'Bleeding Gums United Healthcare tooth decay oral cancer mouth sores'
doctorTxt = 'Heart Attack HealthFirst bleeding gums brain surgery splinter'
doctorVec = removeStopWords([removeStem(doctorTxt)])


################

customerFreq = vec.fit_transform(customerVec)
customerDf = pd.DataFrame(customerFreq.toarray(),columns=vec.get_feature_names())
customerDf

,am,bleed,cover,from,gum,healthcar,suffer,unit,with
0,1,1,1,1,1,1,1,1,1


In [55]:
doctorFreq = vec.fit_transform(doctorVec)
doctorDf = pd.DataFrame(doctorFreq.toarray(),columns=vec.get_feature_names())
doctorDf

,attack,bleed,brain,gum,healthfirst,heart,splinter,surgeri
0,1,1,1,1,1,1,1,1


In [56]:
combinedDf = pd.concat([customerDf, doctorDf],sort=False).fillna(value=0.0)
combinedDf

,am,bleed,cover,from,gum,healthcar,suffer,unit,with,attack,brain,healthfirst,heart,splinter,surgeri
0,1.0,1,1.0,1.0,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,1,0.0,0.0,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [57]:
combinedDf = pd.concat([customerDf, doctorDf],sort=False).fillna(value=0.0)
customerVec = combinedDf.iloc[0].values
doctorVec = combinedDf.iloc[1].values
print(customerVec)
print(doctorVec)
    
    

[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


In [58]:
calcCosSim(customerVec,doctorVec)

0.2357022603955158

In [3]:
resPara = []
resScore = []

splitNum = 300
#for paragraph in contractTxt.split('\n'):
for paragraph in [contractTxt[i:i+splitNum] for i in range(0, len(contractTxt), splitNum)]:
    paragraph = paragraph.translate(string.punctuation)


    try:
        vec = CountVectorizer()
        contractFreq = vec.fit_transform([paragraph])
    except:
        continue
    complaintDf = pd.DataFrame(a.toarray(),columns=complantCnt.get_feature_names())
    contractDf = pd.DataFrame(contractFreq.toarray(),columns=vec.get_feature_names())

    combinedDf = pd.concat([complaintDf, contractDf],sort=False).fillna(value=0.0)
    complainVec = combinedDf.iloc[0].values
    contractVec = combinedDf.iloc[1].values

    simRes = calcCosSim(complainVec,contractVec)
    #print(simRes)
    #print('---------------------')
    resPara.append(paragraph)
    resScore.append(simRes)



#print(resPara[np.argmax(resScore)])
#print('')
#print(np.max(resScore))



rankAr = np.asarray(resPara).argsort()[::-1][:5]

heapLi = []


print('Total number of parts in contract: ', len(contractTxt.split('\n')))
      
print('\n\n')
for ind in rankAr:    
    heapq.heappush(heapLi,[-resScore[ind],ind])
    
while len(heapLi) > 0:
    score,ind = heapq.heappop(heapLi)
    score = score*-1
    
    print(resPara[ind])
    print('')
    print(score)
    print('--------------------------\n')

NameError: ignored